Importing relevant libraries

In [1]:
import pandas as pd
import numpy as np

In [58]:
df1=pd.read_csv('reviews.csv')
df2=pd.read_excel('listings.xlsx')

In [59]:
new_rev = df1[['listing_id','comments']]
new_lst = df2[['id','review_scores_rating']]
df3=pd.merge(new_lst,new_rev, left_on='id', right_on='listing_id', how='left')
df3 = df3[['id','comments','review_scores_rating']]
train_df=df3.dropna()
train_df.head()

,id,comments,review_scores_rating
0,66856,The host canceled my reservation 9 days before...,0.0
1,3947515,The host canceled this reservation 147 days be...,0.0
2,6729959,The host canceled this reservation 40 days bef...,0.0
3,6748779,The host canceled this reservation the day bef...,0.0
4,7278811,The host canceled this reservation 2 days befo...,0.0


In [60]:
train_df['ratings']=train_df['review_scores_rating']//1

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8120/99325947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ratings']=train_df['review_scores_rating']//1


In [61]:
train_df.head()

,id,comments,review_scores_rating,ratings
0,66856,The host canceled my reservation 9 days before...,0.0,0.0
1,3947515,The host canceled this reservation 147 days be...,0.0,0.0
2,6729959,The host canceled this reservation 40 days bef...,0.0,0.0
3,6748779,The host canceled this reservation the day bef...,0.0,0.0
4,7278811,The host canceled this reservation 2 days befo...,0.0,0.0


In [62]:
train_df['ratings']=train_df['ratings'].astype(int)

C:\Users\lenovo\AppData\Local\Temp/ipykernel_8120/1817466627.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['ratings']=train_df['ratings'].astype(int)


In [63]:
train_df['ratings'].unique()

array([0, 1, 2, 3, 4, 5])

In [64]:
X=train_df['comments']
Y=train_df['ratings']

In [65]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)

In [66]:
print("X_train_shape",X_train.shape)
print("X_test",X_test.shape)
print("Y_train",Y_train.shape)
print("Y_test",Y_test.shape)

X_train_shape (316692,)
X_test (79174,)
Y_train (316692,)
Y_test (79174,)


In [73]:
X_train[5]

'The host canceled this reservation 11 days before arrival. This is an automated posting.'

In [74]:
#converting words to index
from keras.preprocessing.text import Tokenizer

In [75]:
tokenizer=Tokenizer(num_words=1000,lower=True)#num words--top frequent words needed
tokenizer.fit_on_texts(X_train)

In [76]:
X_train=tokenizer.texts_to_sequences(X_train)#converting text to numbers
X_test=tokenizer.texts_to_sequences(X_test)

In [77]:
vocabSize=len(tokenizer.word_index)+1
vocabSize

95007

In [78]:
print(X_train[8])#sequence of numbers
#first number 168 means this word is 168th most frequent word in the dataset

[168, 52, 49, 10, 1, 49, 18, 15, 683, 2, 133, 342, 819, 358, 211, 28, 675, 1, 388, 132, 59, 248, 3, 479]


In [79]:
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [80]:
maxWordLen=100 #maximum words taken from each sentence is 100,if it is underlength,padded with 0
X_train=pad_sequences(X_train,padding='post',maxlen=maxWordLen)
X_test=pad_sequences(X_test,padding='post',maxlen=maxWordLen)

In [81]:
X_train[3]

array([ 73, 216,  36,  52,  48, 107,  47,   4, 245,  98, 599,  21,  59,
         3,  54,  43, 259,   1, 136,  40,  19,   5, 311,  47,   4, 170,
        71,  28,  48,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0])

In [84]:
from keras.utils import to_categorical
num_classes=6
Y_train=to_categorical(Y_train,num_classes)
Y_test=to_categorical(Y_test,num_classes)

In [87]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,SimpleRNN
from keras import optimizers


In [90]:
#as RNN expects 3d input

In [88]:
X_train=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],1)
print(X_train.shape)#100 columns as we had padded with 0s and fixed word count as 100 

(316692, 100, 1)


In [89]:
X_test=np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],1)
print(X_test.shape)

(79174, 100, 1)


In [116]:
num_classes=6
def RNNModel():
    model=Sequential()
    model.add(SimpleRNN(50,input_shape=(maxWordLen,1),return_sequences=False))
    model.add(Dense(num_classes))
    model.add(Activation('Softmax'))
    model.summary()
    adam=optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
    return model
    

In [117]:
from keras.wrappers.scikit_learn import KerasClassifier
model=KerasClassifier(build_fn=RNNModel,epochs=5,batch_size=50)


C:\Users\lenovo\AppData\Local\Temp/ipykernel_8120/3371070767.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=RNNModel,epochs=5,batch_size=50)


In [118]:
model.fit(X_train,Y_train)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 50)                2600      
                                                                 
 dense_9 (Dense)             (None, 6)                 306       
                                                                 
 activation_9 (Activation)   (None, 6)                 0         
                                                                 
Total params: 2,906
Trainable params: 2,906
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
6334/6334 [==============================] - 129s 20ms/step - loss: 0.2110 - accuracy: 0.9512
Epoch 2/5
6334/6334 [==============================] - 112s 18ms/step - loss: 0.2073 - accuracy: 0.9519
Epoch 3/5
6334/6334 [==============================] - 108s 17ms/step - loss: 0.2073 - accuracy: 0.9519
Epo

In [120]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
Y_test_=np.argmax(Y_test,axis=1)


2475/2475 [==============================] - 14s 6ms/step


In [121]:
print("Accuracy Score ",accuracy_score(y_pred,Y_test_))

Accuracy Score  0.9522317932654659


In [122]:
#predicting on New Dataset
sent=['Great clean place to stay']
sent=tokenizer.texts_to_sequences(sent)#converting text to numbers
sent=np.array(sent)
sent=pad_sequences(sent,padding='post',maxlen=maxWordLen)
sent=np.array(sent).reshape(sent.shape[0],sent.shape[1],1)

pred=model.predict(np.array(sent))
print(pred)#indicating the ordinal scale from 0 to 5 with 0 being the worst and 5 being the best rating

1/1 [==============================] - 0s 51ms/step
[4]
